Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [142]:
batch_size = 128
hidden_size= 1024
layers = 2
l2_factor = 0.0000001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Layer 1---------------------------------------------------------------
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
  logits_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), 0.5)
  valid_prediction_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  # End of layer 1 -------------------------------------------------------
    
  logits = logits_1
  valid_prediction = valid_prediction_1
  test_prediction = test_prediction_1
  l2_val = l2_factor * tf.nn.l2_loss(weights_1)
    
  for j in range(layers):
    weights = tf.Variable(tf.truncated_normal([hidden_size, hidden_size]))
    biases = tf.Variable(tf.zeros([hidden_size]))
    
    logits = tf.nn.dropout(tf.nn.relu(tf.matmul(logits, weights) + biases), 0.5)
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, weights) + biases)
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, weights) + biases)
    
    l2_val += l2_factor * tf.nn.l2_loss(weights)
  
    
  weights_final = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
  biases_final = tf.Variable(tf.zeros([num_labels]))
    
  l2_val = l2_val + l2_factor * tf.nn.l2_loss(weights_final)
    
  # Training computation.
  logits = tf.matmul(logits, weights_final) + biases_final
  # End of layer 2 -------------------------------------------------------

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_val
  # Optimizer.
    
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0004, global_step,1000,0.90,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_prediction, weights_final) + biases_final)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction, weights_final) + biases_final)

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  #session = tf.Session(config=tf.ConfigProto(log_device_placement=True))
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 402566.843750
Minibatch accuracy: 6.2%
Validation accuracy: 22.5%
Minibatch loss at step 500: 5657.366699
Minibatch accuracy: 55.5%
Validation accuracy: 78.0%
Minibatch loss at step 1000: 1866.000610
Minibatch accuracy: 58.6%
Validation accuracy: 72.2%
Minibatch loss at step 1500: 1203.527466
Minibatch accuracy: 50.8%
Validation accuracy: 65.9%
Minibatch loss at step 2000: 513.052979
Minibatch accuracy: 62.5%
Validation accuracy: 62.7%
Minibatch loss at step 2500: 354.263153
Minibatch accuracy: 55.5%
Validation accuracy: 58.5%
Minibatch loss at step 3000: 447.318420
Minibatch accuracy: 50.0%
Validation accuracy: 56.1%
Minibatch loss at step 3500: 227.442657
Minibatch accuracy: 55.5%
Validation accuracy: 53.5%
Minibatch loss at step 4000: 274.714722
Minibatch accuracy: 49.2%
Validation accuracy: 50.6%
Minibatch loss at step 4500: 209.629852
Minibatch accuracy: 47.7%
Validation accuracy: 48.8%
Minibatch loss at step 5000: 113.949646
Minibatch accurac

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8555254648947417261, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2258683865839528113
 physical_device_desc: "device: XLA_CPU device"]

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
